In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [2]:
import findspark
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

import time as t 
from datetime import date, datetime, timedelta
import joblib
from joblib import Parallel, delayed

import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re
import random

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
# import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
import csv

import gc  # import gorbage collector to resolve the problem of restarting kernel due to large table of population loading in RAM to append
from pyspark.sql.functions import year, col

from collections import defaultdict
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder

import builtins

findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .config("spark.ui.port", "4050") \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "40g")\
       .config("spark.driver.memory", "140g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


#both works
# 1: 
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
# 2:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/22 16:54:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data(problem).csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")


In [4]:
threshold = 0.80

In [5]:
# Get all column names from the population table
all_columns = spark.table("population").columns

# Select only columns that contain "KOLA" and exclude "CHASSIS_ID"
kola_columns = [c for c in all_columns if "KOLA" in c and c != "CHASSIS_ID"]

# Build the main DataFrame with selected columns
df_population = spark.table("population") \
    .select(
        col("VIN"),       
        col("ENGINE_SIZE"),
        col("ENGINE_HP"),
        col("VEH_TYPE"),
        year("VEH_ASSEMB_DATE").alias("VEH_ASSEMB_YEAR"),
        
        *[col(c) for c in kola_columns]           # Include all KOLA-related columns
    )

In [6]:
selected_for_pca_cols = [c for c in df_population.columns if "VIN" not in c]
not_selected_for_pca_cols = [c for c in df_population.columns if c not in selected_for_pca_cols]

# print(non_kola_cols)
df_non_selected_for_pca = df_population.select(*not_selected_for_pca_cols)
df_selected_for_pca = df_population.select(*selected_for_pca_cols)

df_selected_for_pca_pd = df_selected_for_pca.toPandas()  # convert from PySpark to Pandas
print(f"\n The toPandas() finished!")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(f"\n The toPandas() finished!")
file.close()


 The toPandas() finished!


In [7]:
print("\n 🔄 Converting to categorical data...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines("\n 🔄 Converting to categorical data...")
file.close()

df_selected_for_pca_pd = df_selected_for_pca_pd.astype('category')

print(f"\n The convert to category finished!")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(f"\n The convert to category finished!")
file.close()


 🔄 Converting to categorical data...

 The convert to category finished!


In [8]:
print("\n 🚀 Starting OneHotEncoding transformation...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines("\n 🚀 Starting OneHotEncoding transformation...")
file.close()

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
selected_for_pca_encoded = encoder.fit_transform(df_selected_for_pca_pd)

print(f"\n The OneHotEncoding finished!")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(f"\n The OneHotEncoding finished!")
file.close()

onehot_df = pd.DataFrame(selected_for_pca_encoded, columns=encoder.get_feature_names_out())
print(f"\n ✅ onehot_df dataset shape: {onehot_df.shape}")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(f"\n ✅ onehot_df dataset shape: {onehot_df.shape}")
file.close()


 🚀 Starting OneHotEncoding transformation...

 The OneHotEncoding finished!

 ✅ onehot_df dataset shape: (322178, 7908)


In [9]:
# ===============================================================
# This script reduces the dimensionality of a high-dimensional 
# one-hot encoded dataset by identifying and removing columns 
# that exhibit similar **value change patterns** across rows.
#
# Motivation:
# In one-hot encoded data, many columns may carry redundant 
# information — even if their exact values differ — because they 
# follow similar change patterns across data rows. Comparing 
# columns directly (using `np.all`) is too strict, as it only 
# finds exact duplicates. Instead, we compare the **value 
# patterns** using Hamming similarity.
#
# Step-by-step Overview:
# 1. Each column in the one-hot encoded DataFrame is label-encoded 
#    independently. This transforms categorical strings or binary 
#    encodings into integer codes that still preserve the pattern 
#    of value changes (e.g., A → B → A becomes 0 → 1 → 0).
#
# 2. Pairwise comparison of all columns is done using Hamming 
#    similarity — calculated as the proportion of matching 
#    positions in the encoded column vectors.
#
# 3. If two columns have ≥ 85% similarity in their encoded 
#    patterns, they are grouped together and considered redundant.
#
# 4. From each group of similar columns, only the first (as a 
#    representative) is retained, and the others are dropped.
#
# 5. All non-grouped (unique) columns are also retained.
#
# Final Output:
# A reduced feature set with one representative column per 
# similarity group plus all ungrouped columns. This allows for 
# significant dimensionality reduction while preserving 
# meaningful variation across the dataset.
# ===============================================================

In [10]:
print("\n 🔍 Finding highly correlated feature columns...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines("\n 🔍 Finding highly correlated feature columns...")
file.close()



# Label encode each column individually
print("\n🔄 Label encoding all columns in onehot_df...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines("\n🔄 Label encoding all columns in onehot_df...")
file.close()

label_encoded_df = onehot_df.copy()
for col in label_encoded_df.columns:
    encoder = LabelEncoder()
    label_encoded_df[col] = encoder.fit_transform(label_encoded_df[col])

# Initialize structures
similar_cols = []
grouped = set()
groups = defaultdict(list)

# threshold = 0.85  # 85% similarity threshold

print(f"\n 🔍 Comparing columns for Hamming similarity = {threshold}...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines("\n 🔍 Comparing columns for Hamming similarity...")
file.close()


for i in tqdm(range(label_encoded_df.shape[1])):
    if i in grouped:
        continue
    col_i = label_encoded_df.iloc[:, i].values
    for j in range(i + 1, label_encoded_df.shape[1]):
        if j in grouped:
            continue
        col_j = label_encoded_df.iloc[:, j].values
        similarity = np.mean(col_i == col_j)
        if similarity >= threshold:
            grouped.add(j)
            groups[i].append(j)
    if len(groups[i]) > 0:
        groups[i].insert(0, i)
        similar_cols.append(groups[i])

# Select representative columns
representative_cols = [group[0] for group in similar_cols]
all_grouped = set([item for group in similar_cols for item in group])
all_features = set(range(label_encoded_df.shape[1]))
not_grouped = list(all_features - all_grouped)

# Combine representative and ungrouped features
final_features_idx = representative_cols + not_grouped
reduced_encoded = onehot_df.iloc[:, final_features_idx]  # keep original values for modeling

# Output
print(f"\n Representative columns = {representative_cols}")
print(f"\n Not grouped columns = {not_grouped}")
print(f"\n Final selected columns = {final_features_idx}")
print(f"\n ✅ Reduced features from {onehot_df.shape[1]} to {reduced_encoded.shape[1]}")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(f"\n Representative columns = {representative_cols}")
file.writelines(f"\n Not grouped columns = {not_grouped}")
file.writelines(f"\n Final selected columns = {final_features_idx}")
file.writelines(f"\n ✅ Reduced features from {onehot_df.shape[1]} to {reduced_encoded.shape[1]}")
file.close()



 🔍 Finding highly correlated feature columns...

🔄 Label encoding all columns in onehot_df...

 🔍 Comparing columns for Hamming similarity = 0.8...


100%|██████████| 7908/7908 [00:34<00:00, 230.00it/s] 



 Representative columns = [0, 1, 3, 41, 52, 60, 67, 95, 136, 166, 168, 213, 239, 297, 315, 342, 346, 370, 384, 391, 399, 405, 408, 469, 474, 496, 500, 722, 773, 852, 855, 885, 919, 941, 945, 960, 962, 1109, 1146, 1170, 1193, 1328, 1425, 1785, 1998, 2000, 2019, 2035, 2073, 2082, 2104, 2267, 2272, 2352, 2354, 2426, 2443, 2518, 2538, 2812, 3077, 3585, 3598, 3609, 3863, 3864, 3965, 4088, 4155, 4165, 4166, 4219, 4220, 4279, 4474, 4555, 4562, 5248, 5358, 5360, 5478, 5996, 6494, 6585, 6586, 6720, 6976, 6978, 7363, 7376, 7399, 7482]

 Not grouped columns = [6658, 2051, 3078, 3593, 11, 14, 5646, 1552, 3603, 4630, 2583, 1560, 2584, 4631, 5656, 6683, 1565, 2599, 2093, 3630, 2613, 5173, 5174, 5176, 5177, 6718, 5698, 7235, 6223, 4178, 5207, 5215, 5220, 5223, 5226, 7799, 4216, 635, 2177, 1163, 140, 141, 1167, 2711, 1694, 6304, 6305, 1190, 167, 4268, 4783, 1200, 4784, 4790, 2236, 4291, 4805, 5833, 7890, 4314, 2786, 1256, 5353, 1259, 3824, 3825, 2303, 2818, 2307, 2823, 2318, 2319, 6944, 5413, 294, 74

In [11]:
# Log group details to text output
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.write("\n\n📦 Similar Column Groups (Hamming similarity ≥ 85%):\n")
for group in similar_cols:
    rep = group[0]
    file.write(f"Representative column {rep} represents columns {group}\n")
file.close()

# Save group details to CSV

csv_output_path = f"/storage/home/yqf5148/work/volvoPennState/code/similar_column_groups({int(threshold * 100)}% similarity threshold).csv"
with open(csv_output_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Representative_Column_Index", "Group_Column_Indices"])
    for group in similar_cols:
        writer.writerow([group[0], group])
        
print(f"\n📁 Similar column groups saved to: {csv_output_path}")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines("\n 🔍 Comparing columns for Hamming similarity...")
file.close()

# Summary stats
total_columns = onehot_df.shape[1]
num_groups = len(similar_cols)
# grouped_columns = sum(len(g) for g in similar_cols) 
# grouped_columns = sum([len(g) for g in similar_cols])
grouped_columns = builtins.sum([len(g) for g in similar_cols])
  

ungrouped_columns = len(not_grouped)
final_columns = len(final_features_idx)

summary_text = (
    f"\n🔢 Total Columns: {total_columns}"
    f"\n📊 Number of Similarity Groups Found: {num_groups}"
    f"\n🔁 Total Grouped Columns: {grouped_columns}"
    f"\n🔓 Ungrouped Columns: {ungrouped_columns}"
    f"\n✅ Final Feature Count After Reduction: {final_columns}"
)

print(summary_text)
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(summary_text)
file.close()

# Plot: Distribution of group sizes

group_sizes = [len(group) for group in similar_cols]

plt.figure(figsize=(10, 5))
plt.bar(range(len(group_sizes)), group_sizes)
plt.xlabel("Group Index")
plt.ylabel("Number of Columns in Group")
plt.title("📊 Similar Column Group Sizes")
plt.tight_layout()
group_bar_plot_path = f"/storage/home/yqf5148/work/volvoPennState/code/similar_column_group_sizes({int(threshold * 100)}% similarity threshold).png"
plt.savefig(group_bar_plot_path)
plt.close()

print(f"\n 📊 Group size bar plot saved to: {group_bar_plot_path}")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(f"\n 📊 Group size bar plot saved to: {group_bar_plot_path}")
file.close()

# Plot: Histogram of group size frequencies
plt.figure(figsize=(8, 5))
plt.hist(group_sizes, bins=range(1, builtins.max(group_sizes) + 2), edgecolor='black')
plt.xlabel("Group Size")
plt.ylabel("Frequency")
plt.title("📈 Histogram of Similar Column Group Sizes")
plt.tight_layout()
group_hist_plot_path = f"/storage/home/yqf5148/work/volvoPennState/code/group_size_histogram({int(threshold * 100)}% similarity threshold).png"
plt.savefig(group_hist_plot_path)
plt.close()

print(f"\n 📈 Group size histogram saved to: {group_hist_plot_path}")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(f"\n 📈 Group size histogram saved to: {group_hist_plot_path}")
file.close()



📁 Similar column groups saved to: /storage/home/yqf5148/work/volvoPennState/code/similar_column_groups(80% similarity threshold).csv

🔢 Total Columns: 7908
📊 Number of Similarity Groups Found: 92
🔁 Total Grouped Columns: 7766
🔓 Ungrouped Columns: 142
✅ Final Feature Count After Reduction: 234


/tmp/ipykernel_1528460/4028784883.py:56: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_1528460/4028784883.py:58: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.savefig(group_bar_plot_path)



 📊 Group size bar plot saved to: /storage/home/yqf5148/work/volvoPennState/code/similar_column_group_sizes(80% similarity threshold).png


/tmp/ipykernel_1528460/4028784883.py:72: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_1528460/4028784883.py:74: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(group_hist_plot_path)



 📈 Group size histogram saved to: /storage/home/yqf5148/work/volvoPennState/code/group_size_histogram(80% similarity threshold).png


In [12]:
print("\n 🔄 Re-applying PCA on reduced features...")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines("\n 🔄 Re-applying PCA on reduced features...")
file.close()

pca = PCA(n_components=0.95)
pca_applied = pca.fit_transform(reduced_encoded)

print("\n ✅ PCA complete.")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines("\n ✅ PCA complete.")
file.close()


 🔄 Re-applying PCA on reduced features...

 ✅ PCA complete.


In [13]:
pca_columns = [f"PCA_{i+1}" for i in range(pca_applied.shape[1])]
df_pca_applied = pd.DataFrame(pca_applied, columns=pca_columns, index=df_selected_for_pca_pd.index)

In [14]:
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines("\n 🔄 Converting non-selected Spark DataFrame to Pandas...")
file.close()

df_non_selected_for_pca_pd = df_non_selected_for_pca.toPandas()
df_final = pd.concat([df_non_selected_for_pca_pd.reset_index(drop=True), df_pca_applied], axis=1)

print(f"\n ✅ Final dataset shape after concatenation: {df_final.shape}")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(f"\n ✅ Final dataset shape after concatenation: {df_final.shape}")
file.close()


 ✅ Final dataset shape after concatenation: (322178, 136)


In [15]:
output_path = f"/storage/home/yqf5148/work/volvoPennState/data/dataset/final_features_with_pca({int(threshold * 100)}% similarity threshold).csv"
df_final.to_csv(output_path, index=False)

print(f"\n ✅ Final dataset saved to: {output_path}")
file = open(f"/storage/home/yqf5148/work/volvoPennState/code/dimentionalityReductionOutput({int(threshold * 100)}% similarity threshold).txt", "a")
file.writelines(f"\n ✅ Final dataset saved to: {output_path}")
file.close()


 ✅ Final dataset saved to: /storage/home/yqf5148/work/volvoPennState/data/dataset/final_features_with_pca(80% similarity threshold).csv
